In [3]:
%matplotlib inline
import pickle
import copy
import numpy as np
import random
import os
import sys
sys.path.append('..')
from test_utils import *
from ipywidgets import interact
import io
import PIL

In [4]:
import getpass

user = getpass.getuser()
alphapose_root = '/home/{}/codes/AlphaPose'.format(user)
aihub_root = '/home/{}/Datasets/HAAI/AIHUB'.format(user)
fit3d_root = '/home/{}/Datasets/HAAI/Fit3D'.format(user)
ap_aihub_result_root = alphapose_root + "/examples/aihub_result"
ap_fit3d_result_root = alphapose_root + "/examples/fit3d_result"

In [5]:
def draw_skeleton_2d(points, connections, elevation=0, azimuth=0):
    fig = go.Figure()
    
    # Add points
    for point in points.values():
        fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]],
                                   mode='markers', marker=dict(size=2, color='blue')))

    # Add connections
    for connection in connections:
        p1, p2 = points[connection[0]], points[connection[1]]
        fig.add_trace(go.Scatter(x=[p1[0], p2[0]], y=[p1[1], p2[1]],
                                   mode='lines', line=dict(color='black')))

    img_width = 1920
    img_height = 1080
    scale_factor = 0.5

    # https://wikidocs.net/185955 참고
    fig.update_xaxes(range=[-1, 1])
    fig.update_yaxes(range=[1, -1])
    #fig.update_xaxes(autorange="reversed") # 축 범위 반전, 위의 range 설정이 초기화되버림

    fig.update_layout(width=700,height=700)
    
    fig.update_layout(template="plotly_white")

    fig.show()

### GT

In [6]:
gt_root = gt_json = os.path.join(fit3d_root, 's03', 'joints3d_25')
gt_squat_path = os.path.join(gt_root, "squat.json")
gt_squat = readJSON(gt_squat_path)
# gt_squat.keys() # dict_keys(['joints3d_25'])


FileNotFoundError: [Errno 2] No such file or directory: '/home/hrai/Datasets/HAAI/Fit3D/s03/joints3d_25/squat.json'

In [7]:
# gt
fit3d_3d = copy.deepcopy(np.array(gt_squat['joints3d_25'][1025]))
pos = copy.deepcopy(fit3d_3d)
points_aihub_3d = {}
for i, p in enumerate(pos):
    #print(i, h36m_keypoints[i], p)
    points_aihub_3d[i] = np.array([p[0], p[1], p[2]]) #np.array([p[0], p[1], p[2]])  #np.array([p[0], -p[2], -p[1]])
pos_vector = pos.reshape(-1) # (51,)
total_min, total_max = pos_vector.min(), pos_vector.max()
print("total min: ", total_min,
      "total max: ", total_max)

fig = draw_skeleton(points_aihub_3d, fit3d_connections)
fig.show()

total min:  -0.23738800000000002 total max:  1.13802


In [19]:
h36m_connections = [
    (0,1),
    (0,4),
    (0,7),
    (1,2),
    (2,3),
    (4,5),
    (5,6),
    (7,8),
    (8,9),
    (9,10),
    (8,14),
    (14,15),
    (15,16),
    (8,11),
    (11,12),
    (12,13)
]

# gt
fit3d_3d = copy.deepcopy(np.array(gt_squat['joints3d_25'][frame_num]))
pos = copy.deepcopy(fit3d_3d[:17, :])
print(pos.shape)
points_aihub_3d = {}
for i, p in enumerate(pos):
    #print(i, h36m_keypoints[i], p)
    points_aihub_3d[i] = np.array([p[0], p[1], p[2]]) #np.array([p[0], p[1], p[2]])  #np.array([p[0], -p[2], -p[1]])
pos_vector = pos.reshape(-1) # (51,)
total_min, total_max = pos_vector.min(), pos_vector.max()
print("total min: ", total_min,
      "total max: ", total_max)

interact(draw_skeleton(points_aihub_3d, h36m_connections))

(17, 3)
total min:  -0.23738800000000002 total max:  1.13802


interactive(children=(Output(),), _dom_classes=('widget-interact',))

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

### Calculate MPJPE

In [28]:
output.shape

(17, 3)

# ---------------------------------------------------------------------

### Save video

In [6]:
!pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 16.6 MB/s eta 0:00:0000:0100:01


In [5]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
import io
import PIL

r = np.random.RandomState(42)

# sample data
df = pd.DataFrame(
    {
        "step": np.repeat(np.arange(0, 8), 10),
        "x": np.tile(np.linspace(0, 9, 10), 8),
        "y": r.uniform(0, 5, 80),
    }
)

# smaple plotly animated figure
fig = px.bar(df, x="x", y="y", animation_frame="step")

# generate images for each step in animation
frames = []
for s, fr in enumerate(fig.frames):
    # set main traces to appropriate traces within plotly frame
    fig.update(data=fr.data)
    # move slider to correct place
    fig.layout.sliders[0].update(active=s)
    # generate image of current state
    frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png"))))
    
# create animated GIF
frames[0].save(
        "test.gif",
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        duration=500,
        loop=0,
    )

In [11]:
# 2d input video
import plotly.express as px
import pandas as pd
import numpy as np
import io
import PIL

for video in ['s03_60457274_squat', "s03_50591643_squat", "s03_58860488_squat", "s03_65906101_squat"]: # 

    frames = []
    connections = h36m_connections
    actor, cam_num, action = video.split('_')

    for frame_num in range(len(fit3d_result[video]['input'])):

        aihub_2d = copy.deepcopy(fit3d_result[video]['input'][frame_num])

        # 2d skeleton
        pos = copy.deepcopy(aihub_2d)
        points = {}
        for i, p in enumerate(pos):
            p = np.append(p, 0.0)
            #print(i, h36m_keypoints[i], p)
            points[h36m_keypoints[i]] = p

        

        fig = go.Figure()

        # Add points
        for point in points.values():
            fig.add_trace(go.Scatter(x=[point[0]], y=[point[1]],
                                        mode='markers', marker=dict(size=2, color='blue')))

        # Add connections
        for connection in connections:
            p1, p2 = points[connection[0]], points[connection[1]]
            fig.add_trace(go.Scatter(x=[p1[0], p2[0]], y=[p1[1], p2[1]],
                                        mode='lines', line=dict(color='black')))

        # https://wikidocs.net/185955 참고
        fig.update_xaxes(range=[-1, 1])
        fig.update_yaxes(range=[1, -1])
        fig.update_layout(width=700,height=700)
        fig.update_layout(template="plotly_white")

        frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png"))))
        
    # create animated GIF
    frames[0].save(
            "{}_{}_{}_input.gif".format(actor, action, cam_num),
            save_all=True,
            append_images=frames[1:],
            optimize=True,
            duration=33,
            loop=0,
        )

ImportError: Plotly express requires pandas to be installed.

In [7]:
# 3d result video
import plotly.express as px
import pandas as pd
import numpy as np
import io
import PIL


frames = []
connections = h36m_connections

for video in ['s03_60457274_squat', "s03_50591643_squat", "s03_58860488_squat", "s03_65906101_squat"]: # 

    frames = []
    connections = h36m_connections
    actor, cam_num, action = video.split('_')

    for frame_num in range(len(fit3d_result[video]['output'])):
        print(frame_num)
        aihub_3d = copy.deepcopy(fit3d_result[video]['output'][frame_num])
        pos = copy.deepcopy(aihub_3d)
        print(pos.shape)
        # points_aihub_3d = {}
        # for i, p in enumerate(pos):
        #     #print(i, h36m_keypoints[i], p)
        #     points_aihub_3d[h36m_keypoints[i]] = np.array([p[0], p[2], -p[1]]) #np.array([p[0], p[1], p[2]])  #np.array([p[0], -p[2], p[1]])

        connections = [
            (0,1),
            (0,4),
            (0,7),
            (1,2),
            (2,3),
            (4,5),
            (5,6),
            (7,8),
            (8,9),
            (9,10),
            (8,14),
            (14,15),
            (15,16),
            (8,11),
            (11,12),
            (12,13)
        ]

        elevation=20
        azimuth=frame_num*2
        xaxis=dict(range=[-1, 1]) 
        yaxis=dict(range=[-1, 1]) 
        zaxis=dict(range=[-1, 1]) 
        camera=None
        
        fig = go.Figure()
    
        # Add origin
        fig.add_trace(go.Scatter3d(x=[0.0], y=[0.0], z=[0.0],
                                    mode='markers', marker=dict(size=5, color='red')))

        # Add points
        for point in pos:
            fig.add_trace(go.Scatter3d(x=[point[0]], y=[point[2]], z=[-point[1]],
                                    mode='markers', marker=dict(size=2, color='blue')))

        # Add connections
        for connection in connections:
            p1, p2 = pos[connection[0]], pos[connection[1]]
            fig.add_trace(go.Scatter3d(x=[p1[0], p2[0]], y=[p1[2], p2[2]], z=[-p1[1], -p2[1]],
                                    mode='lines', line=dict(color='black')))

        # Set layout
        fig.update_layout(scene=dict(xaxis=xaxis, yaxis=yaxis, zaxis=zaxis,
                                    aspectratio=dict(x=1, y=1, z=1),
                                    camera=dict(eye=dict(x=np.cos(np.pi * azimuth / 180) * np.cos(np.pi * elevation / 180),
                                                        y=np.sin(np.pi * azimuth / 180) * np.cos(np.pi * elevation / 180),
                                                        z=np.sin(np.pi * elevation / 180)))),
                        width=700, height=700, autosize=False)
                        #scene_camera_eye=dict(x=1, y=1, z=1))
        
        if camera is not None:
            fig.update_layout(scene_camera=camera)
        
        
        #fig = draw_skeleton(points_aihub_3d, h36m_connections)

        frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png"))))
        
    # create animated GIF
    frames[0].save(
            "{}_{}_{}_output.gif".format(actor, action, cam_num),
            save_all=True,
            append_images=frames[1:],
            optimize=True,
            duration=33,
            loop=0,
        )

0
(17, 3)
1
(17, 3)
2
(17, 3)
3
(17, 3)
4
(17, 3)
5
(17, 3)
6
(17, 3)
7
(17, 3)
8
(17, 3)
9
(17, 3)
10
(17, 3)
11
(17, 3)
12
(17, 3)
13
(17, 3)
14
(17, 3)
15
(17, 3)
16
(17, 3)
17
(17, 3)
18
(17, 3)
19
(17, 3)
20
(17, 3)
21
(17, 3)
22
(17, 3)
23
(17, 3)
24
(17, 3)
25
(17, 3)
26
(17, 3)
27
(17, 3)
28
(17, 3)
29
(17, 3)
30
(17, 3)
31
(17, 3)
32
(17, 3)
33
(17, 3)
34
(17, 3)
35
(17, 3)
36
(17, 3)
37
(17, 3)
38
(17, 3)
39
(17, 3)
40
(17, 3)
41
(17, 3)
42
(17, 3)
43
(17, 3)
44
(17, 3)
45
(17, 3)
46
(17, 3)
47
(17, 3)
48
(17, 3)
49
(17, 3)
50
(17, 3)
51
(17, 3)
52
(17, 3)
53
(17, 3)
54
(17, 3)
55
(17, 3)
56
(17, 3)
57
(17, 3)
58
(17, 3)
59
(17, 3)
60
(17, 3)
61
(17, 3)
62
(17, 3)
63
(17, 3)
64
(17, 3)
65
(17, 3)
66
(17, 3)
67
(17, 3)
68
(17, 3)
69
(17, 3)
70
(17, 3)
71
(17, 3)
72
(17, 3)
73
(17, 3)
74
(17, 3)
75
(17, 3)
76
(17, 3)
77
(17, 3)
78
(17, 3)
79
(17, 3)
80
(17, 3)
81
(17, 3)
82
(17, 3)
83
(17, 3)
84
(17, 3)
85
(17, 3)
86
(17, 3)
87
(17, 3)
88
(17, 3)
89
(17, 3)
90
(17, 3)
91
(17, 3

In [63]:
pos[0]

array([1.8885511e-04, 5.5102282e-05, 7.1125734e-04], dtype=float32)

In [64]:
connections

[('Pelvis', 'R_Hip'),
 ('Pelvis', 'L_Hip'),
 ('Pelvis', 'Torso'),
 ('R_Hip', 'R_Knee'),
 ('R_Knee', 'R_Ankle'),
 ('L_Hip', 'L_Knee'),
 ('L_Knee', 'L_Ankle'),
 ('Torso', 'Neck'),
 ('Neck', 'Nose'),
 ('Nose', 'Head'),
 ('Neck', 'R_Shoulder'),
 ('R_Shoulder', 'R_Elbow'),
 ('R_Elbow', 'R_Wrist'),
 ('Neck', 'L_Shoulder'),
 ('L_Shoulder', 'L_Elbow'),
 ('L_Elbow', 'L_Wrist')]